# Scraper Monkey

* This project performs web-scraping on Amazon.com, Bestbuy.com using Python to find best price for Samsung TVs.

In [1]:
#import all the necessary libraries:
from bs4 import BeautifulSoup as bs
import pandas as pd
import re
import numpy as np
import requests
import csv
from splinter import Browser
from sqlalchemy import create_engine

In [2]:
#To configure webdriver to use Chrome browser, we have to set the path to chromedriver

##Windows Machine
# executable_path = {"executable_path": "chromedriver.exe"}
# driver = Browser("chrome", **executable_path, headless=False)
#driver= webdriver.Chrome(executable_path)

## Mac Machine
executable_path = {'executable_path': '/usr/local/bin/chromedriver'}
driver = Browser('chrome', **executable_path, headless=False)

In [3]:
product_price_df = pd.DataFrame()
product_price_df["sold_by"] = np.nan
product_price_df["seller_name"] = np.nan
product_price_df["product_Brand"] = np.nan
product_price_df["product_name"] = np.nan
product_price_df["product_model"] = np.nan
product_price_df["product_sku"]=np.nan
product_price_df["product_price"] = np.nan
product_price_df["product_category"]= np.nan
keyword = "Samsung TV"

query_URL_BestBuy_keyword= "https://www.bestbuy.com/site/searchpage.jsp?_dyncharset=UTF-8&id=pcat17071&iht=y&keys=keys&ks=960&list=n&sc=Global&st="+keyword+"&type=page&usc=All%20Categories"

#query_URL_FryElec_keyword= "https://www.frys.com/search?query_search=&cat=-68520&nearbyStoreName=false&pType=pDisplay&fq=a%20Regular%20Items-m10099998%20Samsung&start=0&cat=-68520&from=0&to=99&isKeyword=true"


In [4]:
def extract_products_from_amazon(product_price_df, keyword):
    #Url of the page to be scraped
    request_url = f"https://www.amazon.com/s?k={keyword}"
    print(request_url)
    driver.visit(request_url)

#     for x in range(1, 4):
    response = driver.html
    soup = bs(response)
    for product_doc in soup.findAll('div',attrs={'class':'s-include-content-margin s-border-bottom'}):
        try:
            name=product_doc.find('span', attrs={'class':'a-size-medium a-color-base a-text-normal'})
            product_name = name.text
            if("Samsung" in product_name ):
                price=product_doc.find('span', attrs={'class':'a-offscreen'})
                product_price = price.text
                data = {
                    'product_name': product_name,
                    'product_price': product_price,
#                     'product_model': product_model,
                    'sold_by': 'Amazon',
                    'seller_name': 'Amazon',
                    'product_Brand': 'Samsung TV',
                    'product_category': 'Electronics'
                }
                product_price_df = product_price_df.append(data, ignore_index=True)
        except Exception as e:
            print(f"Oops!  exception occured - continue")
#             browser.click_link_by_partial_text('Next')
    print(product_price_df.count())
#     product_price_df.head()
    return product_price_df

def transformProductPriceAmazon(product_price_df):
# product_price_df

    for row in product_price_df.itertuples():
        try:
            name = product_price_df['product_name'][row.Index]
            print(name)

            pattern = re.compile(r'([UQ]N[0-9a-zA-Z]{5,12})', re.IGNORECASE)
            result = pattern.findall(name)
            for model in result:
    #                 product_model = model
                product_price_df['product_model'][row.Index] =  model

        except IndexError:
            print(f"Oops!  That was no valid number !!")

    return product_price_df 

In [5]:
def scrapProductPriceBestBuy(query_URL_BestBuy_keyword):
    driver.visit(query_URL_BestBuy_keyword)
    content = driver.html
    soup = bs(content)
    row_index=1
    for bestBuyHTML in soup.findAll('li', attrs={'class':'sku-item'}):
        name=bestBuyHTML.find('div', attrs={'class':'sku-title'})
        model = bestBuyHTML.find('div', attrs={'class':'sku-model'})   
        price= bestBuyHTML.find('div', attrs={'class':'priceView-customer-price'})
        model = model.text
        TV_model_name = name.text
        TV_Model = model.split(" ")[0].split(":")[1]
        TV_SKU = model.split(" ")[1].split(":")[1]
        TV_price = price.span.text
        #TV_price = TV_price.split("Your")[0]
        product_price_df.loc[row_index,"sold_by"] = "Best Buy"
        product_price_df.loc[row_index,"seller_name"] = "Best Buy"
        product_price_df.loc[row_index,"product_Brand"] = "Samsung TV"
        product_price_df.loc[row_index,"product_category"] = "Electronics"
        product_price_df.loc[row_index,"product_name"] = TV_model_name.strip()
        product_price_df.loc[row_index,"product_price"] = TV_price
        product_price_df.loc[row_index,"Model_SKU Name"] = model.strip()
        row_index = row_index + 1 
    return product_price_df

def transformProductPriceBestBuy(product_price_df):
    for row in product_price_df.itertuples():
        TV_model_name = product_price_df["Model_SKU Name"][row.Index]
        TV_Model = TV_model_name.split(" ")[0].split(":")[1]
        TV_SKU = TV_model_name.split(" ")[1].split(":")[1]
        product_price_df["product_model"][row.Index] = TV_Model
        product_price_df["product_sku"][row.Index] = TV_SKU
    return product_price_df

In [6]:
# Extract Best Buy product price data
product_price_df= scrapProductPriceBestBuy(query_URL_BestBuy_keyword)
# transform Best Buy product model name to model and SKU
product_price_df = transformProductPriceBestBuy(product_price_df)
product_price_df.head(300)

/Users/m631230/anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel_launcher.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/Users/m631230/anaconda3/envs/PythonData/lib/python3.6/site-packages/pandas/core/indexing.py:202: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_with_indexer(indexer, value)
/Users/m631230/anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel_launcher.py:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returnin

,sold_by,seller_name,product_Brand,product_name,product_model,product_sku,product_price,product_category,Model_SKU Name
1,Best Buy,Best Buy,Samsung TV,"Samsung - 65"" Class - LED - NU6900 Series - 21...",UN65NU6900FXZA,6268405.0,$549.99,Electronics,Model:UN65NU6900FXZA SKU:6268405
2,Best Buy,Best Buy,Samsung TV,"Samsung - 55"" Class - LED - Q60 Series - 2160p...",QN55Q60RAFXZA,6331758.0,$699.99,Electronics,Model:QN55Q60RAFXZA SKU:6331758
3,Best Buy,Best Buy,Samsung TV,"Samsung - 65"" Class - LED - Q70 Series - 2160p...",QN65Q70RAFXZA,6331762.0,"$1,199.99",Electronics,Model:QN65Q70RAFXZA SKU:6331762
4,Best Buy,Best Buy,Samsung TV,"Samsung - 75"" Class - LED - Q70 Series - 2160p...",QN75Q70RAFXZA,6331768.0,"$1,999.99",Electronics,Model:QN75Q70RAFXZA SKU:6331768
5,Best Buy,Best Buy,Samsung TV,"Samsung - 50"" Class - LED - NU6900 Series - 21...",UN50NU6900FXZA,6268403.0,$329.99,Electronics,Model:UN50NU6900FXZA SKU:6268403
6,Best Buy,Best Buy,Samsung TV,"Samsung - 65"" Class - LED - Q80 Series - 2160p...",QN65Q80RAFXZA,6331763.0,"$1,699.99",Electronics,Model:QN65Q80RAFXZA SKU:6331763
7,Best Buy,Best Buy,Samsung TV,"Samsung - 55"" Class - LED - NU6900 Series - 21...",UN55NU6900FXZA,6268404.0,$379.99,Electronics,Model:UN55NU6900FXZA SKU:6268404
8,Best Buy,Best Buy,Samsung TV,"Samsung - 40"" Class - LED - 5 Series - 1080p -...",UN40N5200AFXZA,6360291.0,$199.99,Electronics,Model:UN40N5200AFXZA SKU:6360291
9,Best Buy,Best Buy,Samsung TV,"Samsung - 75"" Class - LED - Q60 Series - 2160p...",QN75Q60RAFXZA,6331767.0,"$1,499.99",Electronics,Model:QN75Q60RAFXZA SKU:6331767
10,Best Buy,Best Buy,Samsung TV,"Samsung - 43"" Class - LED - 7 Series - 2160p -...",UN43RU7100FXZA,6323703.0,$329.99,Electronics,Model:UN43RU7100FXZA SKU:6323703


In [7]:
# Extract Amazon product price data

product_price_df = extract_products_from_amazon(product_price_df, keyword)

product_price_df = transformProductPriceAmazon(product_price_df.copy())
product_price_df.head(20)

https://www.amazon.com/s?k=Samsung TV
Oops!  exception occured - continue
Oops!  exception occured - continue
Oops!  exception occured - continue
sold_by             40
seller_name         40
product_Brand       40
product_name        40
product_model       24
product_sku         24
product_price       40
product_category    40
Model_SKU Name      24
dtype: int64
Samsung - 65" Class - LED - NU6900 Series - 2160p - Smart - 4K UHD TV with HDR
Samsung - 55" Class - LED - Q60 Series - 2160p - Smart - 4K UHD TV with HDR
Samsung - 65" Class - LED - Q70 Series - 2160p - Smart - 4K UHD TV with HDR
Samsung - 75" Class - LED - Q70 Series - 2160p - Smart - 4K UHD TV with HDR
Samsung - 50" Class - LED - NU6900 Series - 2160p - Smart - 4K UHD TV with HDR
Samsung - 65" Class - LED - Q80 Series - 2160p - Smart - 4K UHD TV with HDR
Samsung - 55" Class - LED - NU6900 Series - 2160p - Smart - 4K UHD TV with HDR
Samsung - 40" Class - LED - 5 Series - 1080p - Smart - HDTV
Samsung - 75" Class - LED - Q60 S

/Users/m631230/anaconda3/envs/PythonData/lib/python3.6/site-packages/ipykernel_launcher.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


,sold_by,seller_name,product_Brand,product_name,product_model,product_sku,product_price,product_category,Model_SKU Name
0,Best Buy,Best Buy,Samsung TV,"Samsung - 65"" Class - LED - NU6900 Series - 21...",UN65NU6900FXZA,6268405.0,$549.99,Electronics,Model:UN65NU6900FXZA SKU:6268405
1,Best Buy,Best Buy,Samsung TV,"Samsung - 55"" Class - LED - Q60 Series - 2160p...",QN55Q60RAFXZA,6331758.0,$699.99,Electronics,Model:QN55Q60RAFXZA SKU:6331758
2,Best Buy,Best Buy,Samsung TV,"Samsung - 65"" Class - LED - Q70 Series - 2160p...",QN65Q70RAFXZA,6331762.0,"$1,199.99",Electronics,Model:QN65Q70RAFXZA SKU:6331762
3,Best Buy,Best Buy,Samsung TV,"Samsung - 75"" Class - LED - Q70 Series - 2160p...",QN75Q70RAFXZA,6331768.0,"$1,999.99",Electronics,Model:QN75Q70RAFXZA SKU:6331768
4,Best Buy,Best Buy,Samsung TV,"Samsung - 50"" Class - LED - NU6900 Series - 21...",UN50NU6900FXZA,6268403.0,$329.99,Electronics,Model:UN50NU6900FXZA SKU:6268403
5,Best Buy,Best Buy,Samsung TV,"Samsung - 65"" Class - LED - Q80 Series - 2160p...",QN65Q80RAFXZA,6331763.0,"$1,699.99",Electronics,Model:QN65Q80RAFXZA SKU:6331763
6,Best Buy,Best Buy,Samsung TV,"Samsung - 55"" Class - LED - NU6900 Series - 21...",UN55NU6900FXZA,6268404.0,$379.99,Electronics,Model:UN55NU6900FXZA SKU:6268404
7,Best Buy,Best Buy,Samsung TV,"Samsung - 40"" Class - LED - 5 Series - 1080p -...",UN40N5200AFXZA,6360291.0,$199.99,Electronics,Model:UN40N5200AFXZA SKU:6360291
8,Best Buy,Best Buy,Samsung TV,"Samsung - 75"" Class - LED - Q60 Series - 2160p...",QN75Q60RAFXZA,6331767.0,"$1,499.99",Electronics,Model:QN75Q60RAFXZA SKU:6331767
9,Best Buy,Best Buy,Samsung TV,"Samsung - 43"" Class - LED - 7 Series - 2160p -...",UN43RU7100FXZA,6323703.0,$329.99,Electronics,Model:UN43RU7100FXZA SKU:6323703


In [8]:
product_price_df.count()

sold_by             40
seller_name         40
product_Brand       40
product_name        40
product_model       39
product_sku         24
product_price       40
product_category    40
Model_SKU Name      24
dtype: int64

In [9]:
rds_connection_string = "postgres:{PASSWORD}@localhost:5432/Product_PriceDB"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [10]:
engine.table_names()

['product_price']

In [11]:
product_price_df.to_sql(name='product_price', con=engine, if_exists='replace', index=False)

In [12]:
pd.read_sql_query('select * from product_price', con=engine).head()

,sold_by,seller_name,product_Brand,product_name,product_model,product_sku,product_price,product_category,Model_SKU Name
0,Best Buy,Best Buy,Samsung TV,"Samsung - 65"" Class - LED - NU6900 Series - 21...",UN65NU6900FXZA,6268405.0,$549.99,Electronics,Model:UN65NU6900FXZA SKU:6268405
1,Best Buy,Best Buy,Samsung TV,"Samsung - 55"" Class - LED - Q60 Series - 2160p...",QN55Q60RAFXZA,6331758.0,$699.99,Electronics,Model:QN55Q60RAFXZA SKU:6331758
2,Best Buy,Best Buy,Samsung TV,"Samsung - 65"" Class - LED - Q70 Series - 2160p...",QN65Q70RAFXZA,6331762.0,"$1,199.99",Electronics,Model:QN65Q70RAFXZA SKU:6331762
3,Best Buy,Best Buy,Samsung TV,"Samsung - 75"" Class - LED - Q70 Series - 2160p...",QN75Q70RAFXZA,6331768.0,"$1,999.99",Electronics,Model:QN75Q70RAFXZA SKU:6331768
4,Best Buy,Best Buy,Samsung TV,"Samsung - 50"" Class - LED - NU6900 Series - 21...",UN50NU6900FXZA,6268403.0,$329.99,Electronics,Model:UN50NU6900FXZA SKU:6268403


In [13]:
pd.read_sql_query('select sold_by, count(*) from product_price group by sold_by', con=engine).head()

,sold_by,count
0,Amazon,16
1,Best Buy,24
